## Processamento de Dados

In [37]:
import requests

url = "https://ckan.pbh.gov.br/dataset/ec3efaac-0ca6-4846-9e32-0ffff2d76dbb/resource/4189b28e-7592-474b-8160-e39731315c27/download/20250401_atividade_economica.csv"
output_path = "20250401_atividade_economica.csv"

headers = {
    "User-Agent": "Mozilla/5.0"
}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    with open(output_path, 'wb') as f:
        f.write(response.content)
    print(f"Arquivo salvo como: {output_path}")
else:
    print(f"Erro ao baixar: {response.status_code}")


Arquivo salvo como: 20250401_atividade_economica.csv


In [1]:
import pandas as pd

df = pd.read_csv("20250401_atividade_economica.csv", sep=";", encoding="utf-8")
print(df.columns)
print(f"Registros iniciais: {len(df)}")

Index(['ID_ATIV_ECON_ESTABELECIMENTO', 'CNAE_PRINCIPAL',
       'DESCRICAO_CNAE_PRINCIPAL', 'CNAE', 'DATA_INICIO_ATIVIDADE',
       'NATUREZA_JURIDICA', 'PORTE_EMPRESA', 'AREA_UTILIZADA', 'IND_SIMPLES',
       'IND_MEI', 'IND_POSSUI_ALVARA', 'TIPO_UNIDADE', 'FORMA_ATUACAO',
       'DESC_LOGRADOURO', 'NOME_LOGRADOURO', 'NUMERO_IMOVEL', 'COMPLEMENTO',
       'NOME_BAIRRO', 'NOME', 'NOME_FANTASIA', 'CNPJ', 'GEOMETRIA'],
      dtype='object')
Registros iniciais: 534880


In [2]:
df['DESCRICAO_CNAE_PRINCIPAL'] = df['DESCRICAO_CNAE_PRINCIPAL'].fillna("").str.upper()

keywords = ["BAR", "RESTAURANTE", "BARES", "RESTAURANTES"]

filter = df['DESCRICAO_CNAE_PRINCIPAL'].apply(lambda x: any(k in x for k in keywords))
df_filter = df[filter].copy()

print(f"Registros restantes: {len(df_filter)}")


Registros restantes: 13801


In [8]:
important_data = [
    'NOME', 'NOME_FANTASIA', 'DATA_INICIO_ATIVIDADE', 
    'IND_POSSUI_ALVARA', 'DESC_LOGRADOURO', 
    'NOME_LOGRADOURO', 'NUMERO_IMOVEL', 'NOME_BAIRRO'
]

df_final = df_filter[important_data].copy()

# Usa NOME_FANTASIA se existir, senão usa NOME
df_final['NOME_FINAL'] = df_final['NOME_FANTASIA'].fillna(df_final['NOME'])

# Monta o endereço
df_final['ENDERECO'] = (
    df_final['DESC_LOGRADOURO'].fillna('') + ' ' +
    df_final['NOME_LOGRADOURO'].fillna('') + ', ' +
    df_final['NUMERO_IMOVEL'].fillna('S/N').astype(str) + ' - ' +
    df_final['NOME_BAIRRO'].fillna('')
)

# Seleciona colunas finais
df_final = df_final[['NOME_FINAL', 'DATA_INICIO_ATIVIDADE', 'IND_POSSUI_ALVARA', 'ENDERECO']]
df_final.rename(columns={"NOME_FINAL": "NOME"}, inplace=True)

print(df_final.head())

                                       NOME DATA_INICIO_ATIVIDADE  \
17                 APARECIDA MARIA DE SOUZA            01-07-1993   
19          PIZZARIA E CHURRASCARIA VARANDA            15-10-1993   
29                            RABBIT BURGER            02-05-1994   
172             TATU REI DO ANGU A  BAHIANA            24-11-1993   
196  ROD BITS COMERCIO E REPRESENTACAO LTDA            01-06-1994   

    IND_POSSUI_ALVARA                                           ENDERECO  
17                NÃO  RUA DESEMBARGADOR REIS ALVES, 90 - BAIRRO DAS ...  
19                NÃO                  RUA LUIZ PONGELUPE, 290 - CARDOSO  
29                NÃO                 AVE RESSACA, 118 - PADRE EUSTAQUIO  
172               SIM   RUA DESEMBARGADOR RIBEIRO DA LUZ, 135 - BARREIRO  
196               NÃO              AVE ELIAS ANTONIO ISSA, 288 - LETICIA  


In [9]:
enderecos_unicos = df_final['ENDERECO'].dropna().unique()
print(f"Endereços únicos: {len(enderecos_unicos)}")

Endereços únicos: 11400


## Geocodificação

In [4]:
%pip install geopy

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

geolocator = Nominatim(user_agent="pbh-geocodificador", timeout=5)
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1, max_retries=3, error_wait_seconds=5, swallow_exceptions=False)

In [10]:
def convert_coordinates(address: str):
    try:
        full_address = f"{address}, Belo Horizonte, MG, Brasil"
        location = geocode(full_address)
        if location:
            return pd.Series([location.latitude, location.longitude])
    except Exception as e:
        print(f"[ERRO] '{address}' → {e.__class__.__name__}: {e}")
    return pd.Series([None, None])

In [11]:
df_final[['LATITUDE', 'LONGITUDE']] = df_final['ENDERECO'].apply(convert_coordinates)
print(df_final.head())

RateLimiter caught an error, retrying (0/3 tries). Called with (*('AVE SEBASTIAO DE BRITO, 430 - SANTA ROSA, Belo Horizonte, MG, Brasil',), **{}).
Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/urllib3/connectionpool.py", line 446, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "/usr/lib/python3/dist-packages/urllib3/connectionpool.py", line 441, in _make_request
    httplib_response = conn.getresponse()
  File "/usr/lib/python3.10/http/client.py", line 1375, in getresponse
    response.begin()
  File "/usr/lib/python3.10/http/client.py", line 318, in begin
    version, status, reason = self._read_status()
  File "/usr/lib/python3.10/http/client.py", line 279, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
  File "/usr/lib/python3.10/ssl.py", line 1303, in recv_into
    return self.read(nb

[ERRO] 'AVE PRUDENTE DE MORAIS, 320 - CIDADE JARDIM' → GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=AVE+PRUDENTE+DE+MORAIS%2C+320+-+CIDADE+JARDIM%2C+Belo+Horizonte%2C+MG%2C+Brasil&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=5)"))
                                       NOME DATA_INICIO_ATIVIDADE  \
17                 APARECIDA MARIA DE SOUZA            01-07-1993   
19          PIZZARIA E CHURRASCARIA VARANDA            15-10-1993   
29                            RABBIT BURGER            02-05-1994   
172             TATU REI DO ANGU A  BAHIANA            24-11-1993   
196  ROD BITS COMERCIO E REPRESENTACAO LTDA            01-06-1994   

    IND_POSSUI_ALVARA                                           ENDERECO  \
17                NÃO  RUA DESEMBARGADOR REIS ALVES, 90 - BAIRRO DAS ...   
19               

In [12]:
df_final = df_final.dropna(subset=["LATITUDE", "LONGITUDE"]).reset_index(drop=True)

In [13]:
len(df_final)

9772

In [14]:
df_final.to_csv("bares_restaurantes_geocodificados.csv", index=False, encoding="utf-8")